In [ ]:
!pip install torchsummaryX adamp librosa --quiet

In [ ]:
import librosa
import numpy as np
from torchsummaryX import summary
import sklearn
import sklearn.metrics
import gc
import zipfile
import pandas as pd
import random
from tqdm.autonotebook import tqdm
import os
import datetime
import torch.nn as nn
import torch
from adamp import AdamP
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cpu


In [ ]:
from google.colab import drive # Link your drive if you are a colab user
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
root = '/content/drive/MyDrive/11685/Project/FluteDetection/wav_resampled16kHz'

# Save MFCC files for train, val, test dataset

In [ ]:
# def mfcc_generation(partition, win_length):
#   wav_dir = root + '/' + partition + '/wav/'
#   mfcc_dir = root + '/' + partition + '/mfcc_'+ str(win_length)+'/'
#   wav_files = sorted(os.listdir(wav_dir))
#   for file_name in wav_files:
#     file_path = wav_dir + file_name
#     signal, rate = librosa.load(file_path, sr = None)
#     # sampling rate 16KHz, hop length in seconds: 0.01s, window length in seconds: 0.01s
#     mfcc = librosa.feature.mfcc(y=signal, sr=rate, hop_length=160, window= 'hann',n_mfcc = 20, win_length=win_length, n_fft = 1024, n_mels = 20)
#     mfcc = mfcc.T
#     new_file_name = file_name[:-4] + '.npy'
#     with open(mfcc_dir + new_file_name, 'wb') as f:
#       np.save(f, mfcc)

In [ ]:
# mfcc_generation('train',320)
# mfcc_generation('train', 480)
# mfcc_generation('dev', 320)
# mfcc_generation('dev', 480)
# mfcc_generation('test', 320)
# mfcc_generation('test', 480)

# Save mfcc_delta files for train, val, test dataset

In [ ]:
# def mfcc_delta_generation(partition, win_length):
#   wav_dir = root + '/' + partition + '/wav/'
#   delta_dir = root + '/' + partition + '/mfcc_delta'+'/'
#   wav_files = sorted(os.listdir(wav_dir))
#   for file_name in wav_files:
#     file_path = wav_dir + file_name
#     signal, rate = librosa.load(file_path, sr = None)
#     # sampling rate 16KHz, hop length in seconds: 0.01s, window length in seconds: 0.01s
#     mfcc = librosa.feature.mfcc(y=signal, sr=rate, hop_length=160, window= 'hann',n_mfcc = 20, win_length=win_length, n_fft = 1024, n_mels = 20)
#     mfcc_delta = librosa.feature.delta(mfcc)
#     mfcc_delta = mfcc_delta.T
#     new_file_name = file_name[:-4] + '.npy'
#     with open(delta_dir + new_file_name, 'wb') as f:
#       np.save(f, mfcc_delta)

In [ ]:
# def mfcc_delta2_generation(partition, win_length):
#   wav_dir = root + '/' + partition + '/wav/'
#   delta_dir = root + '/' + partition + '/mfcc_delta_delta'+'/'
#   wav_files = sorted(os.listdir(wav_dir))
#   for file_name in wav_files:
#     file_path = wav_dir + file_name
#     signal, rate = librosa.load(file_path, sr = None)
#     # sampling rate 16KHz, hop length in seconds: 0.01s, window length in seconds: 0.01s
#     mfcc = librosa.feature.mfcc(y=signal, sr=rate, hop_length=160, window= 'hann',n_mfcc = 20, win_length=win_length, n_fft = 1024, n_mels = 20)
#     mfcc_delta2 = librosa.feature.delta(mfcc, order = 2)
#     mfcc_delta2 = mfcc_delta2.T
#     new_file_name = file_name[:-4] + '.npy'
#     with open(delta_dir + new_file_name, 'wb') as f:
#       np.save(f, mfcc_delta2)

In [ ]:
# mfcc_delta_generation('train',160)
# mfcc_delta2_generation('train', 160)
# mfcc_delta_generation('dev', 160)
# mfcc_delta2_generation('dev', 160)
# mfcc_delta_generation('test', 160)
# mfcc_delta2_generation('test', 160)

# Generate labels for annotated onsets

In [ ]:
# def labels_generation(partition):
#   annotations_dir = root + '/' + partition + '/annotations/'
#   labels_dir = root + '/' + partition + '/labels/'
#   mfcc_dir = root + '/' + partition + '/mfcc/'
#   annotation_files = sorted(os.listdir(annotations_dir))
#   for file_name in annotation_files:
#     onsets = []
#     annotation_file_path  = annotations_dir + file_name
#     mfcc_file_path = mfcc_dir + file_name[:-4] + '.npy'
#     length = np.load(mfcc_file_path).shape[0]
#     with open(annotation_file_path, 'rb') as f:
#       lines = f.readlines()
#       for information in lines:
#         information_split = str(information)[2:-5].split("\\t")
#         onset = information_split[1:2]
#         category = information_split[4:5]
#         if category != 'br':
#           onsets += onset
#     f.close()
#     onsets_final = [round(100*float(tt)) for tt in onsets]
#     labels = np.zeros((length, 1))
#     for index in onsets_final:
#       if index < length:
#         labels[index-1:index+1] = 1
#     with open(labels_dir + file_name[:-4] + '.npy', 'wb') as f:
#       np.save(f, labels)

In [ ]:
# labels_generation('train')
# labels_generation('dev')
# labels_generation('test')

# Traditional Method--Energy Based


In [ ]:
def energy_based(signal, threshold, sample_rate): # energy based detection
    pre_emphasis = 0.97
    emphasized_signal = np.append(signal[0],signal[1:] - pre_emphasis * signal[:-1])
    # apply a pre-emphasis filter on the signal
    frame_length = 1024
    frame_step = 512
    NFFT = 2048
    signal_length = len(emphasized_signal)
    num_frames = int(np.ceil(float(np.abs(signal_length-frame_length))/frame_step))
    # make sure that we have at least one frame
    pad_signal_length = num_frames * frame_step + frame_length
    # Pad signal to make sure that all frames have equal number of samples without truncating any samples
    # from the orignal signal
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(emphasized_signal, z)
    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + \
          np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]
    frames *= np.hamming(frame_length)
    mag_frams = np.absolute(np.fft.rfft(frames, NFFT))
    # magnitutude of the FFT
    mag_frams_db = 20 * np.log10(mag_frams)
    # Power Spectrum
    mag_difference = np.zeros((mag_frams.shape[0]-1,mag_frams.shape[1]))
    for i in range(mag_frams.shape[0]-2): # calculate the differences energy
        mag_difference[i,:] = mag_frams[i,:] - mag_frams[i+1,:]
    mag_half = np.clip(mag_difference, 0, None)
    # Half wave rectified
    mag_l2norm = np.zeros((mag_frams.shape[0]-1,1))
    for i in range(mag_frams.shape[0]-2):
        mag_l2norm[i,:]= np.linalg.norm(mag_half[i,:])
    # calculate the L2 norm
    time_1 = np.arange(len(mag_l2norm))
    onset=[]
    for i in range(0, len(mag_l2norm)): # set the threshold and select the onsets
        if mag_l2norm[i,:] >= threshold:
            onset.append(int(np.floor(100*round((i*frame_step/sample_rate),4))))
    return onset


# Test Labels Generation

In [ ]:
def test_labels_generate(path):
    transcript_names = sorted(os.listdir(path))
    transcripts = []
    for name in transcript_names:
      transcript = np.load(path + name)
      transcripts.append(transcript)
    transcripts_final = np.concatenate(transcripts, axis=0)
    return transcripts_final

In [ ]:
wav_test_root = root + '/' + "test" + "/wav/"
mfcc_test_root = root + '/' + "test" + "/mfcc/"
ground_truth = test_labels_generate(root + '/' + "test" + "/labels/")
wav_names = sorted(os.listdir(wav_test_root))
mfcc_names = sorted(os.listdir(mfcc_test_root))
onsets_energy_based = np.zeros((1,1))
onsets_librosa_final = np.zeros((1,1))
threshold = 13
for index in range(len(wav_names)):
  signal, sampling_rate = librosa.load(wav_test_root + wav_names[index], sr = None)
  length = np.load(mfcc_test_root + mfcc_names[index]).shape[0]
  labels_energy = np.zeros((length, 1))
  labels_librosa = np.zeros((length, 1))
  onsets_traditional = energy_based(signal, threshold, sampling_rate)
  onsets_librosa = librosa.onset.onset_detect(y=signal, sr=sampling_rate, units='time')
  onsets_librosa = [round(100*tt) for tt in onsets_librosa]
  for j in onsets_traditional:
    if j < length:
      labels_energy[j-1:j+1] = 1
  for j in onsets_librosa:
    if j < length:
      labels_librosa[j-1:j+1] = 1
  onsets_energy_based = np.concatenate((onsets_energy_based, labels_energy.T), axis=1)
  onsets_librosa_final = np.concatenate((onsets_librosa_final, labels_librosa.T), axis=1)

onsets_librosa_final = onsets_librosa_final[0, 1:].reshape((ground_truth.shape))
onsets_energy_based = onsets_energy_based[0, 1:].reshape((ground_truth.shape))
print(np.count_nonzero(onsets_librosa_final == 1))
print(onsets_energy_based)
test_accuracy = sklearn.metrics.accuracy_score(onsets_energy_based, ground_truth)
test_accuracy1 = sklearn.metrics.accuracy_score(onsets_librosa_final, ground_truth)
auc = sklearn.metrics.roc_auc_score(onsets_librosa_final, ground_truth)
auprc = sklearn.metrics.average_precision_score(onsets_energy_based, ground_truth)
f1 = sklearn.metrics.f1_score(onsets_energy_based, ground_truth)
recall = sklearn.metrics.recall_score(onsets_energy_based, ground_truth)
precision = sklearn.metrics.precision_score(onsets_energy_based, ground_truth)
print("Test Accuracy(Energy-based method): {:.2f}".format(test_accuracy))
print("auc(Energy-based method): {:.2f}".format(auc))
print("auprc(Energy-based method): {:.2f}".format(auprc))
print("f1(Energy-based method): {:.2f}".format(f1))
print("recall(Energy-based method): {:.2f}".format(recall))
print("precision(Energy-based method): {:.2f}".format(precision))
print("Test Accuracy(librosa): {:.2f}".format(test_accuracy1))

<ipython-input-5-0db05d5ab61e>:22: RuntimeWarning: divide by zero encountered in log10
  mag_frams_db = 20 * np.log10(mag_frams)
/usr/local/lib/python3.8/dist-packages/librosa/filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(


2970
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
Test Accuracy(Energy-based method): 0.73
auc(Energy-based method): 0.50
auprc(Energy-based method): 0.18
f1(Energy-based method): 0.15
recall(Energy-based method): 0.13
precision(Energy-based method): 0.17
Test Accuracy(librosa): 0.74


# Train/Validation Dataset

In [ ]:
# Dataset class to load train and validation data

from calendar import day_abbr
from sklearn.preprocessing import normalize


class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, context, offset=0, partition= "train", limit=-1, shuffle=True, Normalization=True): # Feel free to add more arguments

        self.context = context
        self.offset = context
        self.data_path = data_path
        self.normalization = Normalization
        if partition == "train":
            self.mfcc_dir = data_path + '/' + partition + "/mfcc/"
            self.transcript_dir = data_path + '/' + partition + "/labels/"
        else:
            self.mfcc_dir = data_path + '/' + partition + "/mfcc/"
            self.transcript_dir = data_path + '/' + partition + "/labels/"

        mfcc_names = sorted(os.listdir(self.mfcc_dir))
        transcript_names = sorted(os.listdir(self.transcript_dir))
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []

        for i in range(0, len(mfcc_names)):
 
            mfcc = np.load(self.mfcc_dir + mfcc_names[i])
        #   Optionally do Cepstral Normalization of mfcc
        #   Load the corresponding transcript
            transcript = np.load(self.transcript_dir + transcript_names[i])

        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        if partition == "train":
          if shuffle == True:
            Pairs = list(zip(self.mfccs, self.transcripts))
            random.shuffle(Pairs)
            self.mfccs, self.transcripts = zip(*Pairs)

        # Each mfcc is of shape T1 x 20, T2 x 20, ...
        # Each transcript is of shape (T1+2) x 20, (T2+2) x 20

        # TODO: Concatenate all mfccs in self.X such that the final shape is T x 20 (Where T = T1 + T2 + ...) 
        self.mfccs = np.concatenate(self.mfccs, axis=0)
        self.length = len(self.mfccs)
        # TODO: Concatenate all transcripts in self.Y such that the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts = np.concatenate(self.transcripts, axis=0)
        # Hint: Use numpy to concatenate
        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
        if context != 0:
            zero_paddings = np.zeros((context, 20))
            up_paded = np.vstack((zero_paddings, self.mfccs))
            down_paded = np.vstack((up_paded, zero_paddings))
            self.mfccs = down_paded


    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        start_index = ind + self.offset - self.context
        
        ## Calculate ending timestep using offset and context (1 line)
        end_index = ind + self.offset + self.context + 1

        frames = self.mfccs[start_index:end_index]
        
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        height = frames.shape[0]
        width = frames.shape[1]
        if self.normalization == True:
          frames = frames - frames.mean(axis=0, keepdims=True)
          # frames_variance = np.var(frames, axis=0)
          # frames = np.divide(frames, np.tile(frames_variance, (height, 1)))
        frames = torch.FloatTensor(frames) # Convert to tensors
        onset = torch.tensor(self.transcripts[ind])       
        
        return frames, onset

# Test Dataset

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.
    def __init__(self, data_path, context, offset=0, limit=-1, Normalization=True): # Feel free to add more arguments

        self.context = context
        self.offset = context
        self.data_path = data_path
        self.mfcc_dir = data_path + '/' + "test" + "/mfcc/"
        self.normalization = Normalization
        mfcc_names = sorted(os.listdir(self.mfcc_dir))
        

        self.mfccs= []

        for i in range(0, len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(self.mfcc_dir + mfcc_names[i])
            self.mfccs.append(mfcc)

        # NOTE:
        # Each mfcc is of shape T1 x 20, T2 x 20, ...
        # Each transcript is of shape (T1+2) x 20, (T2+2) x 20 before removing [SOS] and [EOS]

        self.mfccs = np.concatenate(self.mfccs, axis=0)
        self.length = len(self.mfccs)

        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
        if context != 0:
            zero_paddings = np.zeros((context, 20))
            up_paded = np.vstack((zero_paddings, self.mfccs))
            down_paded = np.vstack((up_paded, zero_paddings))
            self.mfccs = down_paded

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        start_index = ind + self.offset - self.context
        ## Calculate ending timestep using offset and context (1 line)
        end_index = ind + self.offset + self.context + 1
        frames = self.mfccs[start_index:end_index]
        height = frames.shape[0]
        width = frames.shape[1]
        if self.normalization == True:
          frames = frames - frames.mean(axis=0, keepdims=True)

        frames = torch.FloatTensor(frames) # Convert to tensors 

        return frames

In [ ]:
config = {
    'epochs': 5,
    'batch_size' : 512,
    'context' : 10,
    'learning_rate' : 0.001,
    'architecture' : 'medium-cutoff'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

In [ ]:
data_path = root
train_data = AudioDataset(data_path, context = config['context'], offset=0, partition= "train", limit=-1)
val_data = AudioDataset(data_path, context = config['context'], offset=0, partition= "dev", limit=-1, shuffle=False) 
test_data = AudioTestDataset(data_path, context = config['context'], offset=0, limit=-1) 

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 4,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True)

val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                         batch_size=config['batch_size'], pin_memory= True,
                                         shuffle= False)

test_loader = torch.utils.data.DataLoader(test_data, num_workers= 2, 
                                          batch_size=config['batch_size'], pin_memory= True, 
                                          shuffle= False)

print("Batch size: ", config['batch_size'])
print("Context: ", config['context'])
print("Input size: ", (2*config['context']+1)*20)

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  512
Context:  10
Input size:  420
Train dataset samples = 126430, batches = 247
Validation dataset samples = 17865, batches = 35
Test dataset samples = 17977, batches = 36


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
for i, data in enumerate(train_loader):
    frames, onset = data
    onset = torch.squeeze(onset, 1)
    print(frames.shape, onset.shape)
    break

torch.Size([512, 21, 20]) torch.Size([512])


In [ ]:
class Network(torch.nn.Module):

    def __init__(self, context):

        super(Network, self).__init__()

        input_size = 21 #Why is this the case?
        num_classes = 2
        dropout_rate = 0.3
        self.backbone = torch.nn.Sequential(
            torch.nn.Conv1d(input_size, 128, kernel_size=1, padding=1),
            torch.nn.BatchNorm1d(num_features=128),
            torch.nn.GELU(), 
            torch.nn.Conv1d(128, 128, kernel_size=1, padding=1), 
            torch.nn.BatchNorm1d(num_features=128),
            torch.nn.GELU(), 
            torch.nn.Conv1d(128, 128, kernel_size=1, padding=1), 
            torch.nn.BatchNorm1d(num_features=128),
            torch.nn.GELU(), 
            torch.nn.Conv1d(128, 64, kernel_size=1, padding=1), 
            torch.nn.BatchNorm1d(num_features=64),
            torch.nn.GELU(),
            torch.nn.AdaptiveAvgPool1d((1)),
            torch.nn.Flatten()
        )
        self.cls_layer = torch.nn.Linear(in_features=64, out_features=num_classes)          


    def forward(self, x):

        feats = self.backbone(x)
        out = self.cls_layer(feats)


        return out

model = Network(config['context']).to(device)
frames,onsets = next(iter(train_loader))
summary(model, frames.to(device))

                                   Kernel Shape    Output Shape   Params  \
Layer                                                                      
0_backbone.Conv1d_0                [21, 128, 1]  [512, 128, 22]   2.816k   
1_backbone.BatchNorm1d_1                  [128]  [512, 128, 22]    256.0   
2_backbone.GELU_2                             -  [512, 128, 22]        -   
3_backbone.Conv1d_3               [128, 128, 1]  [512, 128, 24]  16.512k   
4_backbone.BatchNorm1d_4                  [128]  [512, 128, 24]    256.0   
5_backbone.GELU_5                             -  [512, 128, 24]        -   
6_backbone.Conv1d_6               [128, 128, 1]  [512, 128, 26]  16.512k   
7_backbone.BatchNorm1d_7                  [128]  [512, 128, 26]    256.0   
8_backbone.GELU_8                             -  [512, 128, 26]        -   
9_backbone.Conv1d_9                [128, 64, 1]   [512, 64, 28]   8.256k   
10_backbone.BatchNorm1d_10                 [64]   [512, 64, 28]    128.0   
11_backbone.

/usr/local/lib/python3.7/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_backbone.Conv1d_0,"[21, 128, 1]","[512, 128, 22]",2816.0,59136.0
1_backbone.BatchNorm1d_1,[128],"[512, 128, 22]",256.0,128.0
2_backbone.GELU_2,-,"[512, 128, 22]",NaN,NaN
3_backbone.Conv1d_3,"[128, 128, 1]","[512, 128, 24]",16512.0,393216.0
4_backbone.BatchNorm1d_4,[128],"[512, 128, 24]",256.0,128.0
5_backbone.GELU_5,-,"[512, 128, 24]",NaN,NaN
6_backbone.Conv1d_6,"[128, 128, 1]","[512, 128, 26]",16512.0,425984.0
7_backbone.BatchNorm1d_7,[128],"[512, 128, 26]",256.0,128.0
8_backbone.GELU_8,-,"[512, 128, 26]",NaN,NaN


In [ ]:
use_amp = True
criterion = torch.nn.CrossEntropyLoss() #Defining Loss function 
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
# optimizer = AdamP(model.parameters(), lr=config['learning_rate']) #Defining Optimizer
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15,20,25,30], gamma=0.1)
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


In [ ]:
torch.cuda.empty_cache()
gc.collect()

53

# Train

In [ ]:
with torch.no_grad():
    for i, data in enumerate(train_loader):
      
        #TODO: 
        # Follow the following steps, and 
        # Add some print statements here for sanity checking

        #1. What values are you returning from the collate function
        #2. Move the features and target to <DEVICE>
        #3. Print the shapes of each to get a fair understanding 
        #4. Pass the inputs to the model
            # Think of the following before you implement:
            # 4.1 What will be the input to your model?
            # 4.2 What would the model output?
            # 4.3 Print the shapes of the output to get a fair understanding 

        # Calculate loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
        # Calculating the loss is not straightforward. Check the input format of each parameter
        x, y= data
        y = torch.squeeze(y, 1).long()
        x = x.to(device)
        y = y.to(device)
        predictions = model(x)
        print(y.shape)
        print(predictions.shape)

        loss = criterion(predictions, y)
        print(f"loss: {loss}")

        break # one iteration is enough

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([512])
torch.Size([512, 2])
loss: 0.7968158721923828


In [ ]:
def train(model, optimizer, criterion, dataloader, epoch):

    model.train()
    total_train_loss = 0.0 #Monitoring Loss
    total_train_acc = 0.0

    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 
    
    for i , (mfccs, onsets) in enumerate(dataloader):
        onsets = torch.squeeze(onsets, 1).long()
        mfccs = mfccs.to(device)
        onsets = onsets.to(device)
        with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=use_amp):
            ### Forward Propagation
            logits = model(mfccs)
            ### Loss Calculation
            loss = criterion(logits, onsets)

        train_loss = loss.item()

        train_acc = float(torch.sum(logits.argmax(axis=1) == onsets) / onsets.shape[0])
        
        total_train_loss += loss.item()

        total_train_acc += float(torch.sum(logits.argmax(axis=1) == onsets) / onsets.shape[0])

        batch_bar.set_postfix(
    acc="{:.04f}%".format(train_acc),
    loss="{:.04f}".format(train_loss),
    lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        ### Initialize Gradients
        optimizer.zero_grad()

        ### Backward Propagation
        scaler.scale(loss).backward()

        scaler.step(optimizer)

        ### Gradient Descent
        scaler.update()

        batch_bar.update()
  
    batch_bar.close()
    total_train_loss /= len(dataloader)
    total_train_acc /= len(dataloader)
    return train_loss, train_acc*100, optimizer.param_groups[0]["lr"]

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 
    phone_true_list = []
    phone_pred_list = []
    total_val_loss = 0.0
    total_val_acc = 0.0

    for i, data in enumerate(dataloader):

        frames, onsets = data
        ### Move data to device (ideally GPU)
        onsets = torch.squeeze(onsets, 1).long()
        frames, onsets = frames.to(device), onsets.to(device) 

        with torch.inference_mode(): # makes sure that there are no gradients computed as we are not training the model now
            ### Forward Propagation
            logits = model(frames)
            loss = criterion(logits, onsets)
        
        
        val_loss = loss.item()

        val_acc = float(torch.sum(logits.argmax(axis=1) == onsets) / onsets.shape[0])
        
        total_val_loss += loss.item()

        total_val_acc += float(torch.sum(logits.argmax(axis=1) == onsets) / onsets.shape[0])

        batch_bar.set_postfix(
    acc="{:.04f}%".format(val_acc),
    loss="{:.04f}".format(val_loss))
        ### Get Predictions
        predicted_phonemes = torch.argmax(logits, dim=1)
        
        ### Store Pred and True Labels
        phone_pred_list.extend(predicted_phonemes.cpu().tolist())
        phone_true_list.extend(onsets.cpu().tolist())
        
        # Do you think we need loss.backward() and optimizer.step() here?
    
        total_val_loss /= len(dataloader)

        total_val_acc /= len(dataloader)
        del frames, onsets, logits
        torch.cuda.empty_cache()
        batch_bar.update()
        
    batch_bar.close()
    total_val_loss /= len(dataloader)

    total_val_acc /= len(dataloader)
    ### Calculate Accuracy
    accuracy = sklearn.metrics.accuracy_score(phone_pred_list, phone_true_list) 
    return accuracy*100, total_val_loss, total_val_acc

In [ ]:
torch.cuda.empty_cache()

best_acc = 0.0 ### Monitor best accuracy in your run

for epoch in range(config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    train_loss, train_acc, learning_rate = train(model, optimizer, criterion, train_loader, epoch)

    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t Learning Rate {:.04f}".format(
    epoch + 1,
    config['epochs'],
    train_acc,
    train_loss,
    learning_rate))


    accuracy, total_val_loss, total_val_acc = eval(model, val_loader)

    print("Validai Acc: {:.4f}\tValidation Loss: {:.2f}%\tAccuracy Score: {:.2f}%".format(total_val_acc, total_val_loss, accuracy))


    ### Save checkpoint if accuracy is better than your current best
    if accuracy >= best_acc:

      ### Save checkpoint with information you want
      torch.save({'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': train_loss,
              'acc': accuracy}, 
        './model_checkpoint.pth')
      
      best_valacc = accuracy
      

    # Is your training time very high? Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
    # Refer - https://pytorch.org/docs/stable/notes/amp_examples.html



Epoch 1/5


Train:   0%|          | 0/247 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')



Epoch 1/5: 
Train Acc 87.0293%	 Train Loss 0.2937	 Learning Rate 0.0010


Train:   0%|          | 0/35 [00:00<?, ?it/s]

...

Train:   0%|          | 0/247 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')



Epoch 5/5: 
Train Acc 85.5649%	 Train Loss 0.3233	 Learning Rate 0.0010


Train:   0%|          | 0/35 [00:00<?, ?it/s]

Validai Loss: 0.0002	Validation Acc: 0.00%	Accuracy Score: 84.95%


In [ ]:
def test(model, test_loader):
  ### What you call for model to perform inference?
  model.eval()

  ### List to store predicted phonemes of test data
  test_predictions = []

  ### Which mode do you need to avoid gradients?
  with torch.no_grad():

      for i, frames in enumerate(tqdm(test_loader)):

          frames = frames.float().to(device)             
          
          output = model(frames)

          predicted_phonemes = torch.argmax(output, axis=1)
          test_predictions.extend(predicted_phonemes.tolist())
          
  return test_predictions

def test_labels_generate(path):
    transcript_names = sorted(os.listdir(path))
    transcripts = []
    for name in transcript_names:
      transcript = np.load(path + name)
      transcripts.append(transcript)
    transcripts_final = np.concatenate(transcripts, axis=0)
    return transcripts_final

In [ ]:
predictions = test(model, test_loader)
ground_truth = test_labels_generate(root + '/' + "test" + "/labels/")
test_accuracy = sklearn.metrics.accuracy_score(predictions, ground_truth) 
print("Test Accuracy: {:.2f}".format(test_accuracy))

  0%|          | 0/36 [00:00<?, ?it/s]

Test Accuracy: 0.86
